#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩
    * _ _init_ _() : 모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현
- 동적 모델
    * container 모듈 중 nn.ModuleList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공
        - layer 인스턴스 요소 사이에 연관성 없음
        - layer 인스턴스 요소는 인덱싱으로 접근

In [16]:
# 모듈 로딩
import torch                                # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 모듈 (손실함수, 활성화함수 등등)
import torch.optim as optimizer             # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchmetrics.regression import R2Score # 회귀성능지표 관련 모듈
from torchmetrics.classification import *   # 분류성능지표 관련 모듈
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈


import pandas as pd                         # 데이터 파일 분석 관련 모듈
from sklearn.model_selection import train_test_split

In [17]:
# 랜덤 고정
torch.manual_seed(10)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    * 목표 : 은닉층의 개수가 동적인 모델
    * 조건
        * 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
        * 은닉층의 개수 동적 + 퍼셉트론 개수 고정 => 은닉층의 개수, 퍼셉트론 수

In [18]:
# ---------------------------------------------------------------------------------
# 모델이름 : DynamicModel
# 부모클래스 : nn.Module
# 매개변수 : in_in, out_out, h_inout, h_cnt
# ---------------------------------------------------------------------------------

In [19]:
class DynamicModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, out_out, h_inout, h_cnt):
        super().__init__()

        self.in_layer=nn.Linear(in_in, h_inout)
        self.h_layers=nn.ModuleList([nn.Linear(h_inout, h_inout) for _ in range(h_cnt)])
        self.out_layer=nn.Linear(h_inout, out_out)
    
    # 학습 진행 콜백 메서드
    def forward(self, x):
        # 입력층
        y=F.relu(self.in_layer(x))         # 0 <= y

        # 은닉층
        for linear in self.h_layers:
            y=F.relu(linear(y))
        
        # 출력층
        return self.out_layer(y)

In [20]:
# 모델 인스턴스 생성
m1=DynamicModel(3,2,5,10)

In [21]:
# 모델 구조 확인
print(m1)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=5, bias=True)
  (h_layers): ModuleList(
    (0-9): 10 x Linear(in_features=5, out_features=5, bias=True)
  )
  (out_layer): Linear(in_features=5, out_features=2, bias=True)
)


In [22]:

# 모델 파라미터 확인
for name, param in m1.named_parameters():
    print(name,param.shape)

in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
h_layers.0.weight torch.Size([5, 5])
h_layers.0.bias torch.Size([5])
h_layers.1.weight torch.Size([5, 5])
h_layers.1.bias torch.Size([5])
h_layers.2.weight torch.Size([5, 5])
h_layers.2.bias torch.Size([5])
h_layers.3.weight torch.Size([5, 5])
h_layers.3.bias torch.Size([5])
h_layers.4.weight torch.Size([5, 5])
h_layers.4.bias torch.Size([5])
h_layers.5.weight torch.Size([5, 5])
h_layers.5.bias torch.Size([5])
h_layers.6.weight torch.Size([5, 5])
h_layers.6.bias torch.Size([5])
h_layers.7.weight torch.Size([5, 5])
h_layers.7.bias torch.Size([5])
h_layers.8.weight torch.Size([5, 5])
h_layers.8.bias torch.Size([5])
h_layers.9.weight torch.Size([5, 5])
h_layers.9.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


- 학습진행

In [23]:
# 임시데이터 생성
dataTS=torch.FloatTensor([[1,3,5],[2,4,6],[3,5,7],[4,6,8]]) # 2D (4,3)
targetTS=torch.FloatTensor([[7,9],[8,10],[9,11],[10,12]]) # 2D (4,2)

In [25]:
# 모델 학습
pre_y=m1(dataTS)

print(pre_y, pre_y.shape, sep='\n')

tensor([[0.1132, 0.7539],
        [0.1132, 0.7539],
        [0.1132, 0.7539],
        [0.1132, 0.7539]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])
